## **Imports**

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
train_dataset = dsets.MNIST(root='/data', train=True, transform = transforms.ToTensor(), download = True)

# Download MNIST dataset, make it trainable, convert to Tensor

0it [00:00, ?it/s]

9920512it [00:01, 8221079.01it/s]                             


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134876.57it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2224147.22it/s]                            
0it [00:00, ?it/s]

Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51391.80it/s]            


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


##**Loading MNIST Test Dataset**


In [0]:
test_dataset = dsets.MNIST(root='/data', train=False, transform = transforms.ToTensor(), download = True)

## **Make Dataset Iterable**

In [0]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
num_epochs


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# **Building Model**

In [0]:
class neuralnet(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
    super(neuralnet, self).__init__()
    # Linear Function
    self.f1 = nn.Linear(input_dim, hidden_dim)
    
    # Non-Linear Function
    self.sigmoid = nn.Sigmoid()
    
    # Linear Function (readout)
    self.f2 = nn.Linear(hidden_dim, output_dim)
    
  def forward(self,x):
    # Linear Function
    out = self.f1(x)
    # Non-Linear Function
    out = self.sigmoid(out)
    # Linear Function (readout)
    out = self.f2(out)
    return out
    
  

Input Dimension:
Size of Image = 28 x 28 = 784

Hidden Dimension = 100

Output Dimension
= 10
[0,1,2,3,4,5,6,7,8,9]



In [0]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = neuralnet(input_dim, hidden_dim, output_dim)

## **Instantiate Loss Class**

**Cross Entropy Loss**

In [0]:
lossfunc = nn.CrossEntropyLoss()

# CrossEntropyLoss() will compute softmax(logistic/softmax functions)
# and compute cross entropy

##**Instantiate Optimizer Class**

In [0]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

##**Parameters**

In [20]:
print(model.parameters())

print(len(list(model.parameters())))

# Hidden Layer Parameters
print(list(model.parameters())[0].size())

# FC 1 Bias Parameters
print(list(model.parameters())[1].size())

# FC 2 Parameters
print(list(model.parameters())[2].size())

# FC 2 Bias Parameters
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x7fc18c4c9db0>
4
torch.Size([100, 784])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])


#**Train Model**

In [21]:
iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Load images as variable
    images = Variable(images.view(-1,28*28))
    labels = Variable(labels)
                      
    # Clear gradients
    optimizer.zero_grad()
                      
    # Forward pass
    outputs = model(images)
                      
    # Calculate Loss: softmax -> cross entropy loss
    loss = lossfunc(outputs, labels)
                      
    # Back prop
    loss.backward()
    
    # Updating Parameters
    optimizer.step()
                      
    
    iter+=1
         
    if iter%500 == 0:
      # Calculate Accuracy
      correct = 0
      total = 0    
      # Iterate through test dataset
      for images, labels in test_loader:
        # Load images to a torch variable
        images = Variable(images.view(-1,28*28))
           
        # Forward pass to get logits/output
        outputs = model(images)
                      
        # Get predictions from max value
        _, predicted = torch.max(outputs.data,1)
                      
        # Total number of labels
        total += labels.size(0)
        
        # Total correct predictions
        correct += (predicted == labels).sum()
        
       
      accuracy = 100 * correct/total
      
      # Print Loss
      print('Iterations: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))
                      
                      

Iterations: 500. Loss: 0.5082769393920898. Accuracy: 86
Iterations: 1000. Loss: 0.4441291093826294. Accuracy: 89
Iterations: 1500. Loss: 0.4148089289665222. Accuracy: 90
Iterations: 2000. Loss: 0.2600734233856201. Accuracy: 91
Iterations: 2500. Loss: 0.3360753655433655. Accuracy: 91
Iterations: 3000. Loss: 0.2839522659778595. Accuracy: 92
